In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt 

import torch

import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import torchvision.transforms as transforms
from torch.utils.data import  Dataset, TensorDataset, DataLoader


import lib.pytorch_trainer as ptt

from src.imgnet_utils import denormalize

from src.data_loader import _create_dataLoader

from src.Dataset import KaggleSafeDriverDataset

from src.plot_utils import (plot_classes, plot_distribution,
                            statistical_analysis_image, classDistribution,
                            plot_metrics,visualize_predictions,
                            plot_cm_train_valid,plot_layers_weight)
      
from src.convnet_models import (MyResNet, MyInception, MyDenseNet)

from src.extractor_utils import (predict, getPrediction,features_saving)

from utils.utils import (create_submission ,metrics2csv)

In [2]:
print("{} GPU's available:".format(torch.cuda.device_count()) )
cpu_count = torch.multiprocessing.cpu_count()
print("\ncpu_count: {}".format(cpu_count))

4 GPU's available:

cpu_count: 24


In [8]:
use_gpu = torch.cuda.is_available()
use_DataParalel= True 
use_CPU= False        

if use_gpu:
    if use_DataParalel: 
        print("Using DataParalel in all {} GPUS".format(torch.cuda.device_count()))
    else:
        print('Using only one GPU') 
if use_CPU:         # http://pytorch.org/tutorials/beginner/former_torchies/parallelism_tutorial.html 
    print("Using {} CPU's".format(cpu_count))

Using DataParalel in all 4 GPUS


In [9]:
if use_gpu:
    !free -h
    !nvidia-smi

              total        used        free      shared  buff/cache   available
Mem:           220G         51G         99G        566M         69G        166G
Swap:            0B          0B          0B
Sat Nov 25 11:51:13 2017       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 384.90                 Driver Version: 384.90                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00003609:00:00.0 Off |                    0 |
| N/A   63C    P0   129W / 149W |   6705MiB / 11439MiB |      4%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla K80           Off  | 00004BDC:0


# Choose dataset 

In [10]:
path2train = "/mnt/home/e209440/data/train" 
path2test = "/mnt/home/e209440/data/test" 

# Images transformation

In [11]:
imagenet_mean = np.array([0.485, 0.456, 0.406])
imagenet_std  = np.array([0.229, 0.224, 0.225])

img_width = img_height=300 #to use InceptionV3 it must img_width and img_height be changed to 300

# Data augmentation and normalization for training 
data_transforms = {
    'train': transforms.Compose([
        transforms.Scale((img_width, img_height)),
        transforms.ToTensor(),
        transforms.Normalize(imagenet_mean, imagenet_std),
    ]),
    'valid': transforms.Compose([
        transforms.Scale((img_width, img_height)),
        transforms.ToTensor(),
        transforms.Normalize(imagenet_mean, imagenet_std),
    ]),
}        

# Creating  Dataset & Dataloader

In [12]:
batch_size = 32
use_only = 0.1 # Use only is the percentage of the full dataset

In [13]:
dsets = {
    'train': KaggleSafeDriverDataset(path2train, transforms=data_transforms['train'],use_only=use_only),
    'valid': KaggleSafeDriverDataset(path2train, transforms=data_transforms['valid'],use_only=use_only, is_val=True, val_size=0.2),
    'test':  KaggleSafeDriverDataset(path2test, transforms=data_transforms['valid'],use_only=use_only, is_test=True),
}

In [14]:
dset_loaders = _create_dataLoader(dsets, batch_size, pin_memory=False, use_shuffle= True)

In [15]:
dset_sizes = {x: len(dsets[x]) for x in ['train','valid', 'test']} 
dset_classes = len(dsets['train'].y)
dset_classes, dset_sizes

# Dataset have much more samples than datatrain ***It comes from the test.zip****

(1794, {'test': 7972, 'train': 1794, 'valid': 448})

# Checking GPU usage

In [16]:
!free -h
!nvidia-smi

              total        used        free      shared  buff/cache   available
Mem:           220G         51G         99G        566M         69G        166G
Swap:            0B          0B          0B
Sat Nov 25 11:51:29 2017       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 384.90                 Driver Version: 384.90                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00003609:00:00.0 Off |                    0 |
| N/A   62C    P0   140W / 149W |   6705MiB / 11439MiB |     42%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla K80           Off  | 00004BDC:0

# Choosing Model

In [17]:
use_resnet = True
use_inception = False
use_denseNet = False

if use_resnet:
    print('Using ResNet model')
    model = MyResNet()
elif use_inception:
    print('Using Inception model')
    model = MyInception()
elif use_denseNet:
    print('Using DenseNet model')    
    model = MyDenseNet()

Using ResNet model


In [18]:
if use_gpu:
    print('Using GPU')# {}'.format(device_id))
    model.cuda()
    convnet = model.mrnc
    densenet = model.mrnd
    
elif use_DataParalel:
    print("Using all GPU's ")
    model = torch.nn.DataParallel(model)
    model.cuda()
    convnet = model.module.mrnc
    densenet = model.module.mrnd
else:
    convnet = model.mrnc
    densenet = model.mrnd

Using GPU


In [18]:
#print(model)

In [19]:
#extract features from images
convOutput_train = predict(dset_loaders['train'], convnet,use_gpu=(use_gpu or use_DataParalel))
convOutput_valid = predict(dset_loaders['valid'], convnet,use_gpu=(use_gpu or use_DataParalel))
convOutput_test = predict(dset_loaders['test'], convnet,use_gpu=(use_gpu or use_DataParalel))

predict: 56/56 ok
predict: 13/13 ok
predict: 249/249 ok


In [20]:
print(convOutput_train['true'].size(), convOutput_train['pred'].size())
print(convOutput_valid['true'].size(), convOutput_valid['pred'].size())
print(convOutput_test['true'].size(), convOutput_test['pred'].size())

torch.Size([1794]) torch.Size([1794, 6400])
torch.Size([448]) torch.Size([448, 6400])
torch.Size([7972]) torch.Size([7972, 6400])


In [21]:
print(convOutput_train['true'].type(), convOutput_train['pred'].type())

torch.cuda.LongTensor torch.cuda.FloatTensor


# Saving features

In [ ]:
path2features= "/mnt/home/e209440/state-farm-distracted-driver-detection/features/" 

In [ ]:
features_saving(path2features,convOutput_train,use_resnet=True) # Saving train features

features_saving(path2features,convOutput_valid,use_resnet=True) # Saving validation features

features_saving(path2features,convOutput_test,use_resnet=True) # Saving test features


# Loading features

In [ ]:
npzfile = np.load(path2data+"ResNetFeatures.npz")
npzfile.files

In [ ]:
npzfile['true'].shape,npzfile['pred'].shape

In [23]:
conv_dset ={
    'train': TensorDataset(convOutput_train['pred'], convOutput_train['true']),
    'valid': TensorDataset(convOutput_valid['pred'], convOutput_valid['true']),
    'test': TensorDataset(convOutput_test['pred'], convOutput_test['true'])
} 

In [24]:
dset_loaders_convnet = _create_dataLoader(conv_dset, batch_size, 
                        pin_memory=False, use_shuffle= True)

In [25]:
dset_convnet_sizes = {x: len(dset_loaders_convnet[x]) for x in ['train','valid', 'test']} 
dset_convnet_sizes

{'test': 250, 'train': 57, 'valid': 14}

In [30]:
path2saveModel = '/mnt/home/r120084/project/models/distractdriver'
#path2saveModel = '/mnt/home/e209440/models/ResNetDistractDriver' 

savebest = ptt.ModelCheckpoint(path2saveModel,reset=True, verbose=1)

In [34]:
loss_fn = nn.CrossEntropyLoss()
num_epochs = 10

optimizer =  optim.Adam(densenet.parameters(), lr=1e-3)
scheduler = StepLR(optimizer, step_size=5, gamma=0.75)

params = {'model' : densenet, 
    'criterion': loss_fn,  
    'optimizer': optimizer, 
    'callbacks': [savebest, ptt.AccuracyMetric(), ptt.PrintCallback()] #ptt.PlotCallback(),
}

In [35]:
trainer = ptt.DeepNetTrainer(use_gpu=use_gpu,**params)

In [36]:
trainer.fit_loader(num_epochs, dset_loaders_convnet['train'], dset_loaders_convnet['valid'])

Start training for 10 epochs
  1:   2.7s   T: 2.68460 0.11873   V: 2.33999 0.10491 best
  2:   1.9s   T: 2.29251 0.13545   V: 2.29239 0.12054 best
  3:   2.0s   T: 2.22337 0.18841   V: 2.14492 0.15402 best
  4:   1.9s   T: 2.19463 0.21014   V: 2.11340 0.19866 best
  5:   1.9s   T: 1.98984 0.26867   V: 1.81790 0.33259 best
  6:   1.9s   T: 1.63621 0.42363   V: 1.56874 0.45312 best
  7:   1.8s   T: 1.24925 0.56020   V: 2.00022 0.32143 
  8:   2.0s   T: 1.02383 0.65217   V: 0.86339 0.72098 best
  9:   1.9s   T: 0.69728 0.76366   V: 1.69234 0.44866 
 10:   1.9s   T: 0.72919 0.74192   V: 0.93895 0.65179 
Best model was saved at epoch 8 with loss 0.86339: /mnt/home/r120084/project/models/distractdriver
Stop training at epoch: 10/10


# Loading best epoch

In [37]:
trainer.load_state(path2saveModel)
densenet = trainer.model

In [31]:
!free -h
!nvidia-smi

              total        used        free      shared  buff/cache   available
Mem:           220G         52G        102G        571M         65G        165G
Swap:            0B          0B          0B
Sat Nov 25 10:43:47 2017       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 384.90                 Driver Version: 384.90                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00003609:00:00.0 Off |                    0 |
| N/A   65C    P0   139W / 149W |   8688MiB / 11439MiB |     94%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla K80           Off  | 00004BDC:0

In [53]:
info = 'ResNet18'
metrics2csv(trainer, info)

done!


In [54]:
train_eval = trainer.evaluate_loader(dset_loaders_convnet['train'])
valid_eval = trainer.evaluate_loader(dset_loaders_convnet['valid'])

evaluate: 56/56 ok
evaluate: 13/13 ok


In [55]:
train_eval, valid_eval

({'losses': 0.6985351240621629}, {'losses': 0.8633901476860046})

In [56]:
#get output of the trained model (only denset)
result_train = predict(dset_loaders_convnet['train'], densenet,use_gpu=use_gpu)
result_valid = predict(dset_loaders_convnet['valid'], densenet,use_gpu=use_gpu)
result_test = predict(dset_loaders_convnet['test'], densenet,use_gpu=use_gpu)

predict: 56/56 ok
predict: 13/13 ok
predict: 249/249 ok


In [ ]:
result_train = getPrediction(result_train)
result_valid = getPrediction(result_valid)
# result_test['pred'] must be an array of probabilities to make the submission

In [ ]:
correct_train = (result_train['true'] == result_train['pred']).sum()
correct_valid = (result_valid['true'] == result_valid['pred']).sum()

In [ ]:
print('Train: ', correct_train, '/', len(conv_dset['train']) )
print('Valid: ', correct_valid, '/', len(conv_dset['valid']) )

In [ ]:
#visualize_predictions(dsets['valid'], result_valid, correct_pred = True)

In [ ]:
#visualize_predictions(dsets['valid'], result_valid, correct_pred = False)

In [ ]:
#plot_cm_train_valid(result_train,result_valid)

In [ ]:
#plot_layers_weight(dsets,img_width=img_width, img_height=img_height,conv_model = model.mrnc,use_gpu=use_gpu)

# Make submission of the Test set

In [ ]:
create_submission(result_test, 'distracted_driver')